# XY16 dynamical decoupling sequence

In [1]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegenerator
except NameError:
    manager.startModule('logic', 'sequencegenerator')
try:
    pulsedmeasurementlogic
except NameError:
    manager.startModule('gui', 'pulsedmeasurementlogic')   
try:
    poimanagerlogic
except NameError:
    manager.startModule('logic', 'poimaganerlogic')    

In [ ]:
# Get the dictionary containing the default parameters for the desired sequence and alter the values as needed.
# generate_params = pulsedmasterlogic.generate_method_params['rabi']
generate_params=OrderedDict()
generate_params['name'] = 'XY16_seq'
generate_params['tau_start'] = 250.0e-9
generate_params['tau_step'] = 1.0e-9
generate_params['num_of_points'] = 5
generate_params['xy16_order'] = 2

# Call the actual generation method using a wrapper method and pass the parameter set
pulsedmasterlogic.generate_predefined_sequence('XY16_seq', generate_params)
# Give it a moment to generate
time.sleep(0.5)

# The created object is a PulseBlockEnsemble instance, a recipe to create a waveform.
# You can get all created PulseBlockEnsembles using this property (keys are ensemble names, values are the object instances):
for ensemble_name in pulsedmasterlogic.saved_pulse_block_ensembles.keys():
    print(ensemble_name)

# The "with_load" flag can be used to also load the waveform into the channels after creating it
pulsedmasterlogic.sample_ensemble('XY16_seq', with_load=True)
# Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
while pulsedmasterlogic.status_dict['sampload_busy']:
    time.sleep(0.1)
    
pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= generate_params['tau_start'] + 
                                           np.arange(generate_params['num_of_points'])*generate_params['tau_step'],
                                           number_of_lasers=2*generate_params['num_of_points'], 
                                           laser_ignore_list=[], 
                                           alternating=False, 
                                           units=('s', 'arb. u.'))
time.sleep(0.5)

pulsedmasterlogic.set_fast_counter_settings(record_length=4.0e-6, number_of_gates=generate_params['number_of_taus'])
time.sleep(0.5)

# You can get a read-only view of the current measurement settings by using this property:
# for setting, value in pulsedmasterlogic.measurement_settings.items():
#     print('{0}:\n  {1}'.format(setting, value))

In [23]:
setup['poi'] = poimanagerlogic.active_poi.get_key()

In [24]:
refocus_poi(setup['poi'])

102573.166667 2.022601613578252e-09


True

In [25]:
do_xy16_refocus(setup['poi'])

102573.166667 5.8023436214176525e-08
102573.166667 5.435970491493217e-09
102573.166667 3.4348622422339327e-09
102573.166667 5.496413193185982e-09
102573.166667 4.125093808692807e-09
102573.166667 2.492936833245978e-09
102573.166667 3.319237234201901e-09
102573.166667 4.561265715566994e-09
2D gaussian fit not successfull
102573.166667 0.0
102573.166667 3.9896537468678435e-09
2D gaussian fit not successfull
102573.166667 0.0
2D gaussian fit not successfull
102573.166667 0.0
102573.166667 1.3122024578377162e-09
102573.166667 6.762035908312511e-09
102573.166667 9.54881483277814e-10
102573.166667 4.068911046054111e-09
102573.166667 7.809979750353498e-10
102573.166667 1.9837165823190066e-09
102573.166667 1.3240684226995431e-08
2D gaussian fit not successfull
102573.166667 0.0
102573.166667 4.9472741265627225e-09
2D gaussian fit not successfull
102573.166667 0.0
2D gaussian fit not successfull
102573.166667 0.0
2D gaussian fit not successfull
102573.166667 0.0
102573.166667 6.943428301643125e

False

In [26]:
# save measurement
pulsedmasterlogic.save_measurement_data('s','XY16-'+str(xy16['order']),True, False)

'G:\\\\data\\\\QudiData\\2018\\07\\20180702\\PulsedMeasurement'

In [3]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [6]:
def refocus_poi(poi=None, max_drift=0.5,min_counts=setup["min_counts"]):
    if poi==None:
        return False
#     pulsedmasterlogic.load_asset_into_channels('Laser_On')
#     while pulsedmasterlogic.status_dict['loading_busy']:
#         time.sleep(0.5)
#     pulsedmasterlogic.toggle_pulse_generator(False)
#     time.sleep(0.5)
    # perform refocus
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.module_state() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.module_state() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

In [7]:
def do_xy16(save_tag='',load_tag=''):
    
    pulsedmasterlogic.do_fit('No Fit')
    tau_arr = xy16['tau_start'] + np.arange(xy16['points']) * xy16['tau_incr']
    # set parameters in analysis tab
    if xy16['alternating']:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, 2*xy16['points'], 100e-6, [],xy16['alternating'])
    else:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, xy16['points'], 100e-6, [],xy16['alternating'])
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length'], 2*xy16['points'])
    pulsedmasterlogic.analysis_interval_changed(3)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < xy16['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data('s','XY16-'+str(xy16['order'])+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

In [8]:
def do_xy16_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    xy16_total_time = xy16['measurement_time']
    xy16_runtime = 0.0
    xy16['measurement_time'] = xy16['refocus_interval']
    end_measure = do_xy16('xy16_refocus','')
    xy16_runtime += xy16['refocus_interval']
    while xy16_total_time > xy16_runtime:
        for i in range(1):
            end_measure = not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_xy16('xy16_refocus', 'xy16_refocus')
        xy16_runtime += xy16['refocus_interval']
        if end_measure:
            break
    xy16['measurement_time'] = xy16_total_time
    write_to_logfile('xy16_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=xy16_runtime)
    time.sleep(2)
    return end_measure